In [1]:
import tkinter as tk
from tkinter import scrolledtext
import json
import pickle
import numpy as np
import random
import tensorflow
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model
from transformers import BertTokenizer
import random
import torch
from transformers import BertForSequenceClassification, BertTokenizer
import pickle
import time
import sys
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
# Load the trained model and tokenizer
model = BertForSequenceClassification.from_pretrained("./model")
tokenizer = BertTokenizer.from_pretrained("./model")

# Move the model to the appropriate device (e.g., GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set the model to evaluation mode
model.eval()

# Initialize the WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
# Load the fitted LabelEncoder
with open("label_encoder.pkl", "rb") as f:
    encoder = pickle.load(f)

In [3]:
# Load the disease data
with open("data.json", "r") as f:
    disease_data = json.load(f)

In [11]:
class ChatbotGUI(tk.Tk):
    def __init__(self):
        with open("data.json") as file:
            self.intents_data = json.load(file)
            self.intents = self.intents_data["intents"]
        tk.Tk.__init__(self)
        self.title("Chatbot")
        self.geometry("500x500")
        self.configure(bg='#ADD8E6')  # change background color

        self.text_box = scrolledtext.ScrolledText(self, state='disabled', fg='#000080', font=('Helvetica', 12))
        self.text_box.pack(pady=15)

        self.entry = tk.Entry(self, fg='#000080', font=('Helvetica', 12))
        self.entry.pack(pady=15)

        self.button = tk.Button(self, text="Send", command=self.process_user_input, bg='#4B0082', fg='white', font=('Helvetica', 12))
        self.button.pack()

        self.symptoms_input = ""
        self.possible_diseases = []
        self.possible_treatments = {}

        self.greet_user()

    def preprocess_input(self, input_text):
        words = nltk.word_tokenize(input_text)
        words = [word for word in words if word not in stop_words]
        words = [lemmatizer.lemmatize(word) for word in words]
        return " ".join(words)

    def extract_symptoms(self, input_text):
        word_list = nltk.word_tokenize(input_text)
        symptoms = [lemmatizer.lemmatize(word.lower()) for word in word_list if word not in stop_words]
        return symptoms

    def predict_intent(self, input_text):
        treatmentss = []
        encoded_inputs = tokenizer(input_text, truncation=True, padding=True, return_tensors="pt")
        input_ids = encoded_inputs["input_ids"].to(device)
        attention_mask = encoded_inputs["attention_mask"].to(device)
        # Perform classification
        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predicted_labels = torch.argmax(logits, dim=1)
        # Convert predicted labels to original classes using the LabelEncoder
        predicted_classes = encoder.inverse_transform(predicted_labels.cpu().numpy())
        # Get the predicted disease
        predicted_disease = predicted_classes[0]
        for intent in disease_data["intents"]:
            if intent["name"].lower() == predicted_disease.lower():
                treatments = intent["treatments"]
                if treatments:
                    for treatment in treatments:
                        treatmentss.append(treatment)
        for intent in disease_data["intents"]:
            if intent["name"].lower() == predicted_disease.lower():
                description = intent["description"]
        max_proba = torch.max(torch.softmax(logits, dim=1)).item()
        return description, treatmentss, predicted_disease, max_proba

    def greet_user(self):
        greetings = ["Hi! How can I assist you today?", "Hello! How can I help you today?"]
        self.write_to_text_box("Chatbot: " + random.choice(greetings))

    def write_to_text_box(self, content):
        self.text_box.configure(state='normal')
        if content.startswith("User: "):
            self.text_box.insert('end', content + '\n', 'user')
        else:
            self.text_box.insert('end', content + '\n', 'chatbot')
        self.text_box.configure(state='disabled')
        self.text_box.tag_config('user', foreground='#008000', font=('Helvetica', 12, 'italic'))  # change color and font for user messages
        self.text_box.tag_config('chatbot', foreground='#800080', font=('Helvetica', 12, 'bold'))  # change color and font for chatbot messages

    def resetting_user_input(self, user_input):
        if "another symptoms" in user_input.lower() or "help me with another set of symptoms" in user_input.lower() or "new set" in user_input.lower() or "new chart" in user_input.lower() or "fresh" in user_input.lower() or "start" in user_input.lower() or "from beginning" in user_input.lower():
            return True
        else:
            return False

    def get_treatment(self, user_input):
        return "treatment" in user_input.lower() or "cure" in user_input.lower() or \
               "treatments" in user_input.lower() or "cures" in user_input.lower() or \
               "what are the treatments" in user_input.lower()

    def is_exit_message(self, user_input):
        exit_words = ["thank", "thanks", "goodbye", "bye", "quit"]
        return any(word in user_input.lower() for word in exit_words)

    def possible_disease_enquiry(self, user_input):
        enquiry_words = ["possible disease","possible diseases","chances","chance","predicted disease","predicted illness","scope","predict"]
        return any(word in user_input.lower() for word in enquiry_words)

    def get_possible_diseases(self, user_input):
        symptoms = self.extract_symptoms(user_input)
        possible_diseases = []

        for intent in self.intents_data["intents"]:
            for symptom in symptoms:
                if symptom in intent["symptoms"]:
                    possible_diseases.append(intent["name"])
                    break

        return possible_diseases

    def get_possible_diseases_and_treatments(self, user_input):
        symptoms = self.extract_symptoms(user_input)
        possible_diseases = []
        treatments = {}
        w, x, y, z = self.predict_intent(user_input)

        for intent in self.intents_data["intents"]:
            for symptom in symptoms:
                if symptom in intent["symptoms"]:
                    possible_diseases.append(intent["name"])
                    treatments[intent["name"]] = intent["treatments"]
                    break

        return y, possible_diseases, x, treatments

    def process_user_input(self):
        user_input = self.entry.get()

        # Write user input to chat window
        self.write_to_text_box("User: " + user_input)        
        
        if self.resetting_user_input(user_input):
            self.symptoms_input = ""
            self.possible_diseases = []
            self.possible_treatments = {}
            self.write_to_text_box("Chatbot: Sure, how can I help you")
            self.entry.delete(0, 'end')
            return

        if self.is_exit_message(user_input):
            self.write_to_text_box("Chatbot: You're welcome! Take care!")
            self.entry.delete(0, 'end')
            self.quit()
            return

        self.symptoms_input += " " + user_input
        description, treatments, predicted_disease, confidence = self.predict_intent(self.symptoms_input)

        if confidence < 0.12:
            if self.possible_disease_enquiry(user_input):
                self.y, self.possible_diseases, self.x, self.possible_treatments = self.get_possible_diseases_and_treatments(self.symptoms_input)
                if len(self.possible_diseases) > 0:
                    self.write_to_text_box(f"Chatbot: The possible diseases based on your symptoms are: {', '.join(self.possible_diseases)}")
                else:
                    self.write_to_text_box("Chatbot: No other predictions as per my knowledge")
                self.entry.delete(0, 'end')
                return

            if self.get_treatment(user_input):
                if len(self.possible_diseases) > 0:
                    self.write_to_text_box(f"Chatbot: The treatments for {self.y} are: {self.x}")
                    for disease in self.possible_diseases:
                        self.write_to_text_box(f"Chatbot: The treatments for {disease} are: {', '.join(self.possible_treatments[disease])}")
                else:
                    self.write_to_text_box("Chatbot: You haven't asked for possible diseases yet. Please ask for possible diseases first.")
                self.entry.delete(0, 'end')
                return

            common_symptoms_response = ["Your symptoms are quite common, could you please provide more details or any other symptom?", 
                                       "That is a very common symptom, provide me with another symptom you are facing","That's a quite common symptom for many diseases, provide me with any another symptom you are having?"]
            sol = random.choice(common_symptoms_response)
            self.write_to_text_box(f"Chatbot: {sol}")
            self.write_to_text_box(f"Confidence: {confidence}")
            self.entry.delete(0, 'end')
            return

        for item in description:
            self.write_to_text_box(" ".join(item))

        self.write_to_text_box(f"Confidence: {confidence}")

        if self.get_treatment(user_input):
            self.write_to_text_box(f"Chatbot: The treatments are: {treatments}")
            self.entry.delete(0, 'end')
            return

        if self.resetting_user_input(user_input):
            self.symptoms_input = ""
            self.possible_diseases = []
            self.possible_treatments = {}
            self.write_to_text_box("Chatbot: Sure, how can I help you")
            self.entry.delete(0, 'end')
            return  

        if self.is_exit_message(user_input):
            self.write_to_text_box("Chatbot: You're welcome! Take care!")
            self.entry.delete(0, 'end')
            self.quit()
            return



if __name__ == "__main__":
    chatbot = ChatbotGUI()
    chatbot.mainloop()